<a href="https://colab.research.google.com/github/ShoAnn/Legal_Document_IR/blob/main/Apply_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [9]:
import pandas as pd
import gdown
from transformers import BertTokenizerFast, BertForTokenClassification
from tqdm import tqdm
import requests
import torch
import re

In [10]:
tokenizer = BertTokenizerFast.from_pretrained('indolem/indobert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

In [2]:
name = "pidana_500.json"
gdown.download("https://drive.google.com/file/d/1oy70SRPR6wO3I7hJWKdtJ-xr8ICtzJHj/view?usp=drive_link",name , fuzzy=True, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1oy70SRPR6wO3I7hJWKdtJ-xr8ICtzJHj
To: /content/pidana_500.json
100%|██████████| 44.6M/44.6M [00:00<00:00, 153MB/s]


'pidana_500.json'

In [3]:
df = pd.read_json(name)
df

,text,text-tags
0,"[PUTUSAN, ., NOMOR, :, 1974, /, Pid, ., Sus, /...","[O, O, O, O, B_VERN, I_VERN, I_VERN, I_VERN, I..."
1,"[PUTUSAN, Nomor, 1368, /, Pid, ., B, /, 2018, ...","[O, O, B_VERN, I_VERN, I_VERN, I_VERN, I_VERN,..."
2,"[PUTUSAN, Nomor, 985, /, Pid, ., Sus, /, 2017,...","[O, O, B_VERN, I_VERN, I_VERN, I_VERN, I_VERN,..."
3,"[PUTUSAN, No, ., 924, /, Pid, ., B, /, 2014, /...","[O, O, O, B_VERN, I_VERN, I_VERN, I_VERN, I_VE..."
4,"[PUTUSAN, Nomor, 68, /, Pid, ., B, /, 2016, /,...","[O, O, B_VERN, I_VERN, I_VERN, I_VERN, I_VERN,..."
...,...,...
495,"[direktori, putusan, mahkam1h, agung, republik...","[O, O, O, O, O, O, O, O, O, B_VERN, I_VERN, I_..."
496,"[putusan, nomor, 374, /, pid, ., b, /, 2015, /...","[O, O, B_VERN, I_VERN, I_VERN, I_VERN, I_VERN,..."
497,"[putusan, nomor, :, 324, /, pid, ., sus, /, 20...","[O, O, O, B_VERN, I_VERN, I_VERN, I_VERN, I_VE..."
498,"[putusan, ., nomor, :, 36, /, p, id, ., b, /, ...","[O, O, O, O, B_VERN, I_VERN, I_VERN, I_VERN, I..."


In [4]:
df = df.drop('text-tags', axis=1)

In [5]:
# handle empties
df['text'] = df.apply(lambda row: [x if x is not None else ' ' for x in row['text']], axis=1)
# concat to string
df['text'] = df["text"].apply(lambda x: " ".join(x))
# split per ';'
df['text'] = df["text"].str.split(';')

df

,text
0,[PUTUSAN . NOMOR : 1974 / Pid . Sus / 2012 / P...
1,[PUTUSAN Nomor 1368 / Pid . B / 2018 / PN Jkt ...
2,[PUTUSAN Nomor 985 / Pid . Sus / 2017 / PN JKT...
3,[PUTUSAN No . 924 / Pid . B / 2014 / PN . Jkt ...
4,[PUTUSAN Nomor 68 / Pid . B / 2016 / PN . JKT ...
...,...
495,[direktori putusan mahkam1h agung republik ind...
496,[putusan nomor 374 / pid . b / 2015 / pn tlg d...
497,[putusan nomor : 324 / pid . sus / 2015 / pn ....
498,[putusan . nomor : 36 / p id . b / 2 0 0 7 / p...


In [11]:
label_all_tokens = True
def align_word_ids(texts):

    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(1)
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(1 if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids

In [12]:
ids_to_labels = {0: 'B_ADVO', 1: 'B_ARTV', 2: 'B_CRIA', 3: 'B_DEFN', 4: 'B_JUDG', 5: 'B_JUDP', 6: 'B_PENA', 7: 'B_PROS', 8: 'B_PUNI', 9: 'B_REGI', 10: 'B_TIMV', 11: 'B_VERN', 12: 'I_ADVO', 13: 'I_ARTV', 14: 'I_CRIA', 15: 'I_DEFN', 16: 'I_JUDG', 17: 'I_JUDP', 18: 'I_PENA', 19: 'I_PROS', 20: 'I_PUNI', 21: 'I_REGI', 22: 'I_TIMV', 23: 'I_VERN', 24: 'O', 25: 'nan'}

In [13]:
class BertModel(torch.nn.Module):

    def __init__(self, num_labels):

        super(BertModel, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained('indolem/indobert-base-uncased', num_labels=num_labels)

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

model = BertModel(len(ids_to_labels))

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
gdown.download(f'https://drive.google.com/uc?id=1Xkv_QpXvR7eEW7NxgkyUlz1lJ-MTDiA_', 'IndoBERT-LegalNER.pth', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1Xkv_QpXvR7eEW7NxgkyUlz1lJ-MTDiA_
To: /content/IndoBERT-LegalNER.pth
100%|██████████| 440M/440M [00:04<00:00, 106MB/s]


'IndoBERT-LegalNER.pth'

In [15]:
model_weights = torch.load('IndoBERT-LegalNER.pth', map_location=torch.device('cpu'))
model.load_state_dict(model_weights)
model.eval()

BertModel(
  (bert): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(31923, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=Tru

In [16]:
def prediction_to_entities(data_token, prediction_label):
  prev_tag = 'O'
  result = {}
  temp = ''
  for i, word in enumerate(data_token):
    if prediction_label[i] != 'O':
      if prev_tag == 'O' and temp != '':
        temp = ''

      if '##' in word:
        temp += word.replace('##', '')

      else:
        temp +=  ' ' + word
    else:
      if temp != "":
        result[prev_tag.replace("I", "B")] = temp.strip()
      temp = ""

    prev_tag = prediction_label[i]

  return result

def predict_sentence(sentence):
    tokens = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
    input_ids = tokens['input_ids'].to('cpu')
    sentence_mask = tokens['attention_mask'].to('cpu')

    logits = model(input_ids, sentence_mask, None)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to('cpu')
    cleaned_logits = logits[0][label_ids != -100]
    prediction = cleaned_logits.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in prediction]

    input_ids_converted = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
    clean_tokens = [word for word in input_ids_converted if word not in ['[CLS]', '[SEP]', '[PAD]']]

    return prediction_to_entities(clean_tokens, prediction_label)


In [22]:
print(df['text'][0][0], "\n Prediksi : ", predict_sentence(df['text'][0][0]), "\n")
print(df['text'][0][1], "\n Prediksi : ", predict_sentence(df['text'][0][1]), "\n")
print(df['text'][0][2], "\n Prediksi : ", predict_sentence(df['text'][0][2]), "\n")

PUTUSAN . NOMOR : 1974 / Pid . Sus / 2012 / PN . JKT . BAR . DEMI KEADILAN BERDASARKAN KETUHANAN YANG MAHA ESA . Pengadilan Negeri Jakarta Barat yang memeriksa dan mengadili perkara - perkara pidana dalam peradilan tingkat pertama dengan acara biasa telah menjatuhkan putusan sebagai berikut atas nama terdakwa : AGNES TRI AHADI Als ANAS , lahir di Jakarta , umur / tanggal , 30 tahun / 8 Agustus 1982 , jenis kelamin laki - laki , kebangsaan Indonesia , tempat tinggal Jl . Kamp . Bali Gg . II . No . 5 Kel . Kampung Bali Kec . Tanah Abang Jakarta Pusat Terdakwa ditahan sejak tanggal 27 Juli 2012 sampai sekarang  
 Prediksi :  {'B_VERN': '1974 / pid . sus / 2012 / pn . jkt . bar .', 'B_DEFN': 'agnes tri ahadi als anas ,'} 

 Terdakwa menolak didampingi oleh Penasehat Hukum  
 Prediksi :  {} 

 Pengadilan Negeri tersebut  
 Prediksi :  {} 

